In [1]:
!pip install transformers

In [6]:
from google.colab import files

uploaded = files.upload()

Saving Reddit_Data.csv to Reddit_Data.csv


In [7]:
import pandas as pd
import numpy as np

df_final=pd.read_csv('Reddit_Data.csv')
df_final.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [8]:
df_final.isnull().sum()

,0
clean_comment,100
category,0


In [9]:
df_final.dropna(subset=['clean_comment','category'],axis=0,inplace=True)

In [10]:
X=list(df_final['clean_comment'])
y=list(df_final['category'])

In [11]:
X

[' family mormon have never tried explain them they still stare puzzled from time time like some kind strange creature nonetheless they have come admire for the patience calmness equanimity acceptance and compassion have developed all the things buddhism teaches ',
 'buddhism has very much lot compatible with christianity especially considering that sin and suffering are almost the same thing suffering caused wanting things shouldn want going about getting things the wrong way christian this would mean wanting things that don coincide with god will and wanting things that coincide but without the aid jesus buddhism could also seen proof god all mighty will and omnipotence certainly christians are lucky have one such christ there side but what about everyone else well many christians believe god grace salvation and buddhism god way showing grace upon others would also help study the things jesus said and see how buddha has made similar claims such rich man getting into heaven joke basic

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
from transformers import DistilBertTokenizerFast
toki=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
train_encodings=toki(X_train,truncation=True,padding=True)
test_encodings=toki(X_test,truncation=True,padding=True)

In [18]:
print(train_encodings)

In [19]:
import tensorflow as tf
train_dataset= tf.data.Dataset.from_tensor_slices((dict(train_encodings),y_train))
test_dataset=tf.data.Dataset.from_tensor_slices((dict(test_encodings),y_test))

In [20]:
X_train_rf = np.array(train_encodings['input_ids'])
X_test_rf = np.array(test_encodings['input_ids'])

In [22]:
print(X_train_rf)

[[  101  1051  9541 ...     0     0     0]
 [  101  2028  2008 ...     0     0     0]
 [  101  8401 25911 ...     0     0     0]
 ...
 [  101 13198  3492 ...     0     0     0]
 [  101  2138 12098 ...     0     0     0]
 [  101  2204  2935 ...     0     0     0]]


In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize the RandomForest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf_model.fit(X_train_rf, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test_rf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.5713324360699865


In [26]:
criterion=['gini','entropy','log_loss']
max_features=['auto','sqrt','log2']
max_depth=[int(x) for x in np.linspace(10,1000,10)]
min_samples_split=[int(x) for x in np.linspace(1,15,15)]
min_samples_leaf=[int(x) for x in np.linspace(1,15,15)]
random_state=[int(x) for x in np.linspace(1,100,100)]


random_grid_1={'criterion':criterion,
            'max_features':max_features,
            'max_depth':max_depth,
            'min_samples_split':min_samples_split,
            'min_samples_leaf':min_samples_leaf,
            'random_state':random_state,
            }
print(random_grid_1)

{'criterion': ['gini', 'entropy', 'log_loss'], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'random_state': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]}


In [27]:
rfr_rdm=RandomForestClassifier()
from sklearn.model_selection import RandomizedSearchCV
rfr_rdm_1=RandomizedSearchCV(estimator=rfr_rdm,param_distributions=random_grid_1,n_iter=100,cv=5,verbose=2,random_state=100,n_jobs=-1)
rfr_rdm_1.fit(X_train_rf,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
210 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
130 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/util

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10, 11,
                                                             12, 13, 14, 15],
                                        'min_samples_split': [1, 2, 3, 4, 5, 6,
                                                              7, 8, 9, 10, 11,
                                                              12, 13, 14, 15],
                                        'random_state': [1, 2, 3, 4, 5, 6, 7, 8,
                                                         9, 10, 11, 12, 13, 14,
                                                         15, 16, 17, 18, 19, 20,
                                                         21, 22, 23, 24, 25, 26,
                                                         27, 28, 29, 30, ...]},
                   random_state=100, verbose=2)

In [28]:
rfr_rdm_1.best_params_

{'random_state': 92,
 'min_samples_split': 14,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 890,
 'criterion': 'entropy'}

In [29]:
rfr_rdm_result=rfr_rdm_1.best_estimator_

In [31]:
rfr_rdm_pred=rfr_rdm_result.predict(X_test_rf)
accuracy1 = accuracy_score(rfr_rdm_pred,y_test)
print(f'Accuracy: {accuracy1}')

Accuracy: 0.5796769851951548
